# chrony 时间同步

在安装k8s集群前需确保各节点时间同步；`chrony` 是一个优秀的 `NTP`
实现，性能比ntp好，且配置管理方便；它既可作时间服务器服务端，也可作客户端。

- `OpenStack` 社区也推荐使用
`chrony`实现各节点之间的时间同步

## 安装配置介绍
项目中选定一个节点(`groups.chrony[0]`)作为集群内部其他节点的时间同步源，而这个节点本身从公网源同步；当然如果整个集群都无法访问公网，那么请手动校准这个节点的时间后，仍旧可以作为内部集群的时间源服务器。
- 配置 chrony server, 在`/etc/chrony.conf` 配置以下几项，其他项默认值即可

In [ ]:
# 1. 配置时间源，国内可以增加阿里的时间源 ntp1.aliyun.com
server {{ ntp_server }} iburst

# 2. 配置允许同步的客户端网段
allow {{ local_network }}

# 3. 配置离线也能作为源服务器
local stratum 10

- 配置 chrony client

In [ ]:
# 1. 清除所有其他时间源，只配置一个本地`groups.chrony[0]`节点作为源
server {{ groups.chrony[0] }} iburst

# 2. 其他所有项可以默认配置

## `kubeasz` 集成安装

- 修改 ansible hosts 文件，在 `chrony`组中加入选中的节点ip
- [可选] 修改
roles/chrony/defaults/main.yml 中的变量定义

对于新集群或者新节点，`chrony`
的安装配置已经集成到脚本中；对于已运行中的集群请执行如下命令进行安装：

`ansible-playbook /etc/ansible/roles/chrony/chrony.yml `

## 验证配置

- 在 chrony server 检查时间源信息，默认配置为`ntp1.aliyun.com`的地址：

In [ ]:
$ chronyc sources -v
210 Number of sources = 9

  .-- Source mode  '^' = server, '=' = peer, '#' = local clock.
 / .- Source state '*' = current synced, '+' = combined , '-' = not combined,
| /   '?' = unreachable, 'x' = time may be in error, '~' = time too variable.
||                                                 .- xxxx [ yyyy ] +/- zzzz
||      Reachability register (octal) -.           |  xxxx = adjusted offset,
||      Log2(Polling interval) --.      |          |  yyyy = measured offset,
||                                \     |          |  zzzz = estimated error.
||                                 |    |           \
MS Name/IP address         Stratum Poll Reach LastRx Last sample               
===============================================================================
^* 120.25.115.20                 2  10   377   999  +2625us[+2691us] +/-   28ms
^+ 203.107.6.88                  2  10   377   32m    -17ms[  -17ms] +/-   40ms
^+ 139.199.215.251               2  10   377  1005  +4464us[+4529us] +/-   45ms
^+ 111.230.189.174               2  10   377   25m  +4786us[+4846us] +/-   72ms
^- sv1.ggsrv.de                  2  10   377   28m  +2533us[+2592us] +/-  163ms
^+ stratum2-1.ntp.led01.ru.>     2  10   377   352    +27ms[  +27ms] +/-   82ms
^- 192.99.68.38                  2  10   377   220    +16ms[  +16ms] +/-  173ms
^- de-user.deepinid.deepin.>     3  10   377   31m  -9939us[-9881us] +/-  180ms
^- ntp.xtom.nl                   2  10   377   29m    -28ms[  -28ms] +/-  131ms

- 在 chrony server 检查时间源同步状态

In [ ]:
chronyc sourcestats -v
210 Number of sources = 9
                             .- Number of sample points in measurement set.
                            /    .- Number of residual runs with same sign.
                           |    /    .- Length of measurement set (time).
                           |   |    /      .- Est. clock freq error (ppm).
                           |   |   |      /           .- Est. error in freq.
                           |   |   |     |           /         .- Est. offset.
                           |   |   |     |          |          |   On the -.
                           |   |   |     |          |          |   samples. \
                           |   |   |     |          |          |             |
Name/IP Address            NP  NR  Span  Frequency  Freq Skew  Offset  Std Dev
==============================================================================
120.25.115.20              12   7  224m     -0.001      0.224  +3504us   778us
203.107.6.88               26  12  448m     -0.101      0.088    -11ms   995us
139.199.215.251            21  13  344m     +0.119      0.123  +4499us   883us
111.230.189.174            15   9  258m     +0.011      0.086  +3219us   365us
sv1.ggsrv.de               33  18   10h     -0.168      0.207  +1866us  3334us
stratum2-1.ntp.led01.ru.>  48  25   13h     -0.156      0.144  +3368us  3978us
192.99.68.38                6   5   86m     -2.030      7.218  -1120us  2636us
de-user.deepinid.deepin.>  24  13  397m     +0.065      0.345  -4791us  3223us
ntp.xtom.nl                56  32   16h     -0.248      0.120    -34ms  4605us

- 在 chrony client 检查，可以看到时间源只有一个（groups.chrony[0] 节点地址）

In [ ]:
chronyc sources
# 输出结果
210 Number of sources = 1
MS Name/IP address         Stratum Poll Reach LastRx Last sample               
===============================================================================
^* notebook.wfh.net              3   9   377    76    +31us[ +115us] +/-   23ms

chronyc sourcestats
# 输出结果
210 Number of sources = 1
Name/IP Address            NP  NR  Span  Frequency  Freq Skew  Offset  Std Dev
==============================================================================
notebook.wfh.net            6   5   38m     +0.041      0.149  +5861ns    35us

## 验证时间同步状态完成

chrony 服务启动后，chrony server 会与配置的公网参考时间服务器进行同步；server 同步完成后，chrony
client 会与 server 进行时间同步；一般来说整个集群达到时间同步需要几十分钟。可以用如下命令检查，初始时 **NTP synchronized:
no**，同步完成后 **NTP synchronized: yes**

In [ ]:
ansible all -m shell -a 'timedatectl'
# 输出结果
192.168.20.202 | CHANGED | rc=0 >>
      Local time: Thu 2019-09-19 10:30:44 CST
  Universal time: Thu 2019-09-19 02:30:44 UTC
        RTC time: Thu 2019-09-19 02:30:44
       Time zone: Asia/Shanghai (CST, +0800)
     NTP enabled: yes
NTP synchronized: yes
 RTC in local TZ: no
      DST active: n/a

192.168.20.201 | CHANGED | rc=0 >>
      Local time: Thu 2019-09-19 10:30:45 CST
  Universal time: Thu 2019-09-19 02:30:45 UTC
        RTC time: Thu 2019-09-19 02:30:45
       Time zone: Asia/Shanghai (CST, +0800)
     NTP enabled: yes
NTP synchronized: yes
 RTC in local TZ: no
      DST active: n/a

192.168.20.203 | CHANGED | rc=0 >>
      Local time: Thu 2019-09-19 10:30:46 CST
  Universal time: Thu 2019-09-19 02:30:46 UTC
        RTC time: Thu 2019-09-19 02:30:46
       Time zone: Asia/Shanghai (CST, +0800)
     NTP enabled: yes
NTP synchronized: yes
 RTC in local TZ: no
      DST active: n/a

 [WARNING]: Unhandled error in Python interpreter discovery for host 192.168.20.206: Failed to connect to the host via ssh: ssh: connect to host
192.168.20.206 port 22: Connection timed out

 [WARNING]: Unhandled error in Python interpreter discovery for host 192.168.20.205: Failed to connect to the host via ssh: ssh: connect to host
192.168.20.205 port 22: Connection timed out

192.168.20.3 | CHANGED | rc=0 >>
      Local time: Thu 2019-09-19 10:30:55 CST
  Universal time: Thu 2019-09-19 02:30:55 UTC
        RTC time: Thu 2019-09-19 02:30:55
       Time zone: Asia/Shanghai (CST, +0800)
     NTP enabled: yes
NTP synchronized: yes
 RTC in local TZ: no
      DST active: n/a

192.168.20.204 | CHANGED | rc=0 >>
      Local time: Thu 2019-09-19 10:30:56 CST
  Universal time: Thu 2019-09-19 02:30:56 UTC
        RTC time: Thu 2019-09-19 02:30:55
       Time zone: Asia/Shanghai (CST, +0800)
     NTP enabled: yes
NTP synchronized: yes
 RTC in local TZ: no
      DST active: n/a

192.168.20.205 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Data could not be sent to remote host \"192.168.20.205\". Make sure this host can be reached over ssh: ssh: connect to host 192.168.20.205 port 22: No route to host\r\n", 
    "unreachable": true
}
192.168.20.206 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Data could not be sent to remote host \"192.168.20.206\". Make sure this host can be reached over ssh: ssh: connect to host 192.168.20.206 port 22: Connection timed out\r\n", 
    "unreachable": true
}